In [223]:
%config IPCompleter.greedy=True
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import re
import datetime


In [ ]:
%lsmagic  # list all the available magic function
%time
%timeit

%matplotlib inline

%%markdown
%%bash
%%HTML
%%python3 | python2 | perl

%who_ls | whos    # list all the variables
%reset # delete all the variables

%pip freeze | egrep --color=always "request|pandas|seaborn"
  
    

In [3]:
%%bash
ls data

Tank_1.xlsx
Tank_2.xlsx
Tank_3.xlsx


In [222]:
files = glob.glob(r"./data/*.xlsx")

print("\n++++++++++++++++++++++++++++++++++++++++++\n")
print("The file list used for combination:\n")
[ print( f" The {n+1} spreadsheet is {file}") for n, file in enumerate(files)]
print("\n++++++++++++++++++++++++++++++++++++++++++\n")


#[print(i, f) for i, f in enumerate(xls)]
#sheets = [key for key in xls]
#workbooks = [xls[key] for key in xls]

def add_name_sheet(sheet_name, worksheet):
    """
     add the name of the worksheet as a column name
     
    """
    #worksheet["sheet_name"] = sheet_name
    worksheet.insert(loc = 0, column = "sheet_name", value = sheet_name)
    
    return worksheet

def combine_all_worksheets(workbook):
    """
    combined all worksheets in a single 
    
    """
    xls = pd.read_excel(workbook, sheet_name = None, usecols = list(range(6)))
    lst = [ add_name_sheet(sheet_name, sheet) for sheet_name, sheet in xls.items()]
    df = pd.concat(lst, axis=0, ignore_index=True, sort=False)
    #df["workbook_xxname"] =  "xxxx"
    workbook_name =  re.findall(r'[^\/]+(?=\.)', workbook)[0]
    df.insert(loc = 0, column = "workbook_name", value = workbook_name)
    
    return df


df_lst = [combine_all_worksheets(file) for file in files]
df_comb =  pd.concat(df_lst, axis=0, ignore_index=True, sort=False)

print(f" There are {df_comb.shape[0]} rows x {df_comb.shape[1]} columns ") 
print("\n++++++++++++++++++++++++++++++++++++++++++\n")

df_comb.iloc[:5, :10]




++++++++++++++++++++++++++++++++++++++++++

The file list used for combination:

 The 1 spreadsheet is ./data/Tank_1.xlsx
 The 2 spreadsheet is ./data/Tank_2.xlsx
 The 3 spreadsheet is ./data/Tank_3.xlsx

++++++++++++++++++++++++++++++++++++++++++

 There are 672 rows x 11 columns 

++++++++++++++++++++++++++++++++++++++++++



,workbook_name,sheet_name,Order #,Samples,Passage,Amount,Cells conc,Freeze date,Freeze\n date,Label#
0,Tank_1,Box1-CordBlood,1,14-4 /#1 ECM(from 2D/noDex),p4,25,3x10^6,2014-05-15 00:00:00,NaT,NaN
1,Tank_1,Box1-CordBlood,2,14-4 /#1 2D(from 2D/noDex),p4,2,3.4x10^6,2014-05-15 00:00:00,NaT,NaN
2,Tank_1,Box2-CordBlood,1,13-9 / #1 from ECM,p4,17,3x10^6,2014-01-06 00:00:00,NaT,NaN
3,Tank_1,Box2-CordBlood,2,13-9 / #1 from ECM,p4,18,5x10^6; 1.5ml,2014-01-06 00:00:00,NaT,NaN
4,Tank_1,Box2-CordBlood,3,13-9 / #1 from ECM,p4,10,5x10^6,2014-01-06 00:00:00,NaT,NaN


In [237]:
from pandas import ExcelWriter

today = datetime.date.today().strftime ('%Y%m%d')
excel_file = "".join(["combined_worksheet_", today, ".xlsx" ])

writer = ExcelWriter(excel_file)
df_comb.to_excel(writer, sheet_name="combination", index = False)
writer.save()